## ETL ANALISIS DESCRIPTIVO HUMEDAD DEL SUELTO BASE DEL SIATA

Carga de librerias para el ETL

In [1]:
import pandas as pd
from sodapy import Socrata
import sqlite3
import os
import calendar
import ephem
import pandas as pd

c:\Users\jofza\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Parametros de nombre de la carpeta y nombre de la tabla a ingestar en base de datos

In [14]:
carpeta = 'Historico_humedad'
nombre_tabla = 'Historico_humedad'

Extraccion de la información manual

In [4]:
def ingesta_aut(carpeta,nombre_tabla):
    contenido = os.listdir(str('./'+ str(carpeta) +'/'))
    df_tbl = pd.DataFrame()
    for i in contenido:
        df = pd.read_csv(r'./'+ str(carpeta) +'/'+str(i), sep= ',', encoding='latin-1')
        df_tbl =  pd.concat([df_tbl, df])
    db_path = 'Data_Base.db'
    conn = sqlite3.connect(db_path)
    df_tbl.to_sql(str('tbl_siata_')+str(nombre_tabla) , conn, if_exists='replace', index=False)
    return print('La ingesta se realizó correctamente a la tabla ' + str (nombre_tabla))

Carga en base de datos local

In [15]:
ingesta_aut(carpeta,nombre_tabla)

Creación de dimensión de tiempo y posterior a esta la creación de nuevas variables exogenas

In [7]:
tbl_dim_tiempo = pd.DataFrame(pd.date_range("2021-09-01","2023-09-01", freq='H'), columns=['fecha_hora']) 
tbl_dim_tiempo['fecha'] = tbl_dim_tiempo.fecha_hora.dt.strftime("%Y-%m-%d").astype(str)
tbl_dim_tiempo['fecha_id'] = tbl_dim_tiempo.fecha_hora.dt.strftime("%Y-%m-%d %H").astype(str)
tbl_dim_tiempo['mes'] = tbl_dim_tiempo.fecha_hora.dt.strftime("%m").astype(str)
tbl_dim_tiempo['dia_mes'] = tbl_dim_tiempo.fecha_hora.dt.strftime("%d").astype(str)
tbl_dim_tiempo['semana'] = tbl_dim_tiempo.fecha_hora.dt.isocalendar().week
tbl_dim_tiempo['dia'] = tbl_dim_tiempo.fecha_hora.dt.day_name(locale='es_CO')
tbl_dim_tiempo.head(5)

C:\Users\jofza\AppData\Local\Temp\ipykernel_5300\3259203753.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  tbl_dim_tiempo = pd.DataFrame(pd.date_range("2021-09-01","2023-09-01", freq='H'), columns=['fecha_hora'])


,fecha_hora,fecha,fecha_id,mes,dia_mes,semana,dia
0,2021-09-01 00:00:00,2021-09-01,2021-09-01 00,09,01,35,Miércoles
1,2021-09-01 01:00:00,2021-09-01,2021-09-01 01,09,01,35,Miércoles
2,2021-09-01 02:00:00,2021-09-01,2021-09-01 02,09,01,35,Miércoles
3,2021-09-01 03:00:00,2021-09-01,2021-09-01 03,09,01,35,Miércoles
4,2021-09-01 04:00:00,2021-09-01,2021-09-01 04,09,01,35,Miércoles


Crea la función para traer la fases de la luna y la iluminación de acuerdo a la fecha

In [11]:
def get_lunar_phase_and_illumination(date):
    observer = ephem.Observer()
    observer.lat, observer.lon = '6.244203', '-75.581215'  # Medellín coordinates
    observer.date = date
    moon = ephem.Moon(observer)
    phase = moon.phase
    illumination = moon.phase

    if phase == 0:
        phase_name = 'Luna nueva'
    elif 0 < phase < 50:
        phase_name = 'Luna creciente'
    elif phase == 50:
        phase_name = 'Primer cuarto'
    elif 50 < phase < 100:
        phase_name = 'Luna menguante'
    elif phase == 100:
        phase_name = 'Luna llena'
    elif 100 < phase < 150:
        phase_name = 'Giboso menguante'
    elif phase == 150:
        phase_name = 'Último cuarto Luna'
    elif 150 < phase < 200:
        phase_name = 'Creciente menguante'
    else:
        phase_name = 'Unknown'
    
    return phase_name, f"{illumination:.2f}"



Integración de variables  dimension de tiempo con fases lunares 

In [8]:
tbl_dim_tiempo['fecha'] = pd.to_datetime(tbl_dim_tiempo['fecha'])
#tbl_dim_tiempo[['fase_lunar', 'iluminacion']] = tbl_dim_tiempo['fecha'].apply(lambda date: pd.Series(get_lunar_phase_and_illumination(date)))


In [9]:
tbl_dim_tiempo.head(5)

,fecha_hora,fecha,fecha_id,mes,dia_mes,semana,dia
0,2021-09-01 00:00:00,2021-09-01,2021-09-01 00,09,01,35,Miércoles
1,2021-09-01 01:00:00,2021-09-01,2021-09-01 01,09,01,35,Miércoles
2,2021-09-01 02:00:00,2021-09-01,2021-09-01 02,09,01,35,Miércoles
3,2021-09-01 03:00:00,2021-09-01,2021-09-01 03,09,01,35,Miércoles
4,2021-09-01 04:00:00,2021-09-01,2021-09-01 04,09,01,35,Miércoles


Carga de nueva dimension de tiempo con variables exogenas a base de datos local

In [11]:
#conexion Base de datos
db_path = 'Data_Base.db'
conn = sqlite3.connect(db_path)

In [12]:

tbl_dim_tiempo.to_sql("tbl_dim_tiempo", conn, if_exists='replace', index=False)

17521